# Agent with Long-Time Memory
* We will build an Agent that will help us to **manage a ToDo list**.
* It will decide:
    * **when to save items** to our ToDo list.
    * **to save either a user profile or a collection of ToDo items**.
* In addition to semantic memory (user facts), it will also have **procedural memory**.
    * Remember, the procedural memory is the system prompt. This will allow the user to set preferences for creating ToDo items.

In [ ]:
#pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ.get("Open_ai_key_here")

Lets install Langchain here...



In [ ]:
#!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenA

chatModel35 = ChatOpenAI(model="gpt-3.5-turbo-0125")
chatModel4o = ChatOpenAI(model = "gpt-4o")



Using TrustCall here....

In [ ]:
from pydantic import BaseModel, Field

class Memory(BaseModel):
    conent: str = Field(description= "The main content of the memory. For example:" \
    "User expressed interest in learning about French language...")

class MemoryCollection(BaseModel):
    memories: list[Memory] = Field(description="A list of memories about the user.")

In [ ]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI

## Everytime you make instance of this class, you make a new list of tools used...

class Spy:
    def __init__(self):
        self.called_tools = []

    def __call__(self, run):
        ### This is here will collect information about the tools used in the extractor...
        q = [run]
        while q:
            r = q.pop()
            if r.child_runs:
                q.extend(r.child_runs)
            if r.run_type == "chat_model":
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                    
                )
spy = Spy()

##Initialize the model here....
model = ChatOpenAI(model="gpt-4o", temperature=0)

## create the extractor here....

trustcall_extractor = create_extractor(
    model,
    tools=[Memory],
    tool_choice= "Memory",
    enable_inserts=True,
)

### Add the spy as a listener to the extractor....

trustcall_extractor_see_all_tool_calls = trustcall_extractor.with_listeners(on_end=spy)



## Running Trustcall without "listener" to monitor the workflow tool call Just to test Trustcall...

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


## Instruction to the model to extract the memories from the user input...
instruction = """Extract memmories from the following converstion 'Mac'..."""

### Converstion here....
conversation =[HumanMessage(content="Hi I'm Chris"),
               AIMessage(content= "Nice to meet you 'Jackass'"),
               HumanMessage(content="Yesterday I visited the local state prison to make fun of all the prisoners...")]

### Using the regular extractor here without the listener....
result = trustcall_extractor.invoke("messages": [SystemMessage(content= instruction)] + conversation})




In [ ]:
### Messages that contain tool calls....

for m in result["messages"]:
    m.pretty_print()

    



In [ ]:
### Responses contain the memories that adhere to the schema....

for m in result["responses"]:
    print(m)

    

In [ ]:
### Metadata contains the tool calls used in the extraction process...

for m in result["response_metadata"]:
    print(m)

In [ ]:
## Update the conversation here...

updated_conversation = [AIMessage(content="Thats great. what did you do after?"),
                        HumanMessage(content="I went to Tiburon and prepared a paella in the park"),
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about finally learning to cook paella for " \
                        "the sake of my girlfriend."),]

## Update the  instruction here...

system_instruction = """Update existing memories and create new ones BASES on the followinf converstaion: """


## We'll save the existing memories, giving them an ID, key (tool name), and value....

tool_name = "Memory"
existing_memories= [(str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result["responses"])] if result["responses"] else None
existing_memories




## And now let's see Trustcall with the listener...

In [ ]:
## Now here, we will use the extractor with the listener now.
## We will envoke the extractor with the updated conversation and existing memories...

result = trustcall_extractor_see_all_tool_calls.invoke({"messages": updated_conversation,
                                                        "existing": existing_memories})



In [ ]:
## Metadata contains the tool call....

for m in result["response_metadata"]:
    print(m)

    

In [ ]:
### Messages ccontain the tool calls here...

for m in result["messages"]:
    m.pretty_print()

In [ ]:
## Parsed responses....

for m in result["responses"]:
    print(m)

In [ ]:
## Inspet the tool calls made by the "Trustcall" extractor....

spy.called_tools

## Ok, now in this section I will create the "UpdateMemory" class to select the element in the long-term memory we will update at one particular moment...

In [ ]:
## first import the "TypedDict" and the "Literal" classes from the "typing" module....
from typing import TypedDict, Literal
## make the UpdateMemory class and pass TypedDict as the base class...

class UpdateMemory(TypedDict):
    """Decision on what memory type to update."""
    update_type:Literal["user", "todo", "instructions"]

    


## Now I will build the actual agent

* I will use the router "route_message" to make a binary decision to save memories.
* The memory collection updating will be handled by "Trustcall" in the write_memory node, like I did previously....

In [ ]:
import uuid
from IPython.display import Image, display

from datetime import datetime
from trustcall import create_extractor
from typing import Optional
from pydantic import BaseModel, Field

from langchain_core.runnables import RunnableConfig
from langchain_core.messages import merge_message_runs, HumanMessage, SystemMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraphm MessagesState, END, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

from langchain_openai import ChatOpenAI

##Initialize the chat model here....

model = ChatOpenAI(model="gpt-4o", temperature=0)


## User Profile class definition here...
"""So this here is the PROFILE of the user you are 'chatting' with..."""

name: Optional[str] = Field(description= "The user's name", default= None)
location: Optional[str] =Field(description="The user's location", default= None)
job: Optional[str] = Field(description="The user's job", default= None)
connections: list[str] = Field(
    description= "Personal connection of the user, such as familiy members, friends, or coworkers",
    default_factory=list

)
interests: list[str] = Field(
    description ="Interests that the user has",
    default_factory= list
)


## This is the "To-Do" Schema here...

class ToDo(BaseModel):
    task: str= Field(description="The task to be completed...")
    time_to_complete: Optional[int] = Field(description= "Estimated time to complete the task (minutes)")
    deadline: Optional[datetime] =Field(
        description="When the task needs to be completed by (if applicable)",
        default = None
        
    )
    solutions: list[str] = Field(
        description = "List of specific, actionable solutions here like (specific ideas, service providers," \
        "or concrete options relevant to completing the task)",
        min_items = 1,
        default_factory=list
    )

    status: Literal["not started", "in progress", "done", "archived"] = Field(
        description="Current status of the task",
        default= "not started"
    )


### create the Trustcall extractor for updating the user profile here...

profile_extractor = create_extractor(
    model,
    tools=[Profile],
    tool_choice="Profile",
)

## Chabot instruction for choosing what to update and what tools to call...

MODEL_SYSTEM_MESSAGE ="""You are a helpful chatbot.
You are designed to be a companion to a user, helping them keep track of
their "TO-DO" list.

You have a 'long-term' memory ability which keeps track of three things...

1. The user's profile (general information about them)
2. The user's "to-do" list
3. General instructions for updating the "to-do" list

Here is the current User Profile (may be empty if no information has been collected yet.):

<user_profil>
{user_profile}
</ser_profile>

Here is the current "to-do" list. (may be empty if no tasks have been added yet):
<todo>
{todo}
</todo>

Here are the current user-specific preferences for updating to ToDo list. (May be empty if
no preferences have been specified yet):
<instructions>
{instructions}
</instructions>

Here are your instructions for reasoning about the user's messages....

1. Reason carefully about th user's messages as presented below.

2. Decide whether any of your long-term memory should be updated:
    - If personal information was provided about the user,
    update the user's profile by calling UpdateMemory tool with type 'user'.
    - If tasks are mentioned, update the ToDo list by calling UpdateMemory tool with type 'todo'.
    - If the user has specific preferences for how to update the ToDo list, update the instructions
    by calling UpdateMemory tool with type 'instructions'.

3. Tell the user that you have updated your memory, if appropirate:
    - Do not tell user you have updated the user's profile.
    - Tell the user then, when you update the todo list.
    - Do not tell the user that you have updated instructions.

4. Err on the side of updating the todo list. No need to ask for explicit permission.

5. Respond naturally to user after a tool call was made to save memories or if no
tool call was made."""

### 
TRUSTCALL_INSTRUCTION = """Reflect in the following interaction...
User the provided tool list to retain any necessary memories about the user.
User parallel tool calling to handle updates and insertions simultaneously.

System Time: {time}"""

### Instructions for updating the ToDo list:

CREATE_INSTRUCTIONS= """Reflect in the following.

Based on this interaction, update your instructions for how to update ToDO list items.
Use any feedback from the user to update how they like tp have items added, etc.

Your current instructions are:

<current_instructions>
{current_instructions}
</current_instructions>"""

### Node definitions...

def task_mAIstro(state: MessagesState, config: RunnableConfig, store: BaseStore):

    """Load memories from the store and use them to personalize the chatbot's response"""

    ##Get the user ID from the config....
    user_id = config["configurable"]["user_id"]

    ### Retrieve profile memory from the store...
    namespace = ("profile", user_id)
    memories = store.search(namespace)
    if memories:
        user_profile = memories[0].value
    else:
        user_profile= None

    ## Retrieve task memory from the store....

    namespace = ("todo", user_id)
    memories = store.search(namespace)
    todo = "\n".join(f"{mem.value}" for mem in memories)

    ## Retrieve custom instructions...

    namespace = ("instructions", user_id)
    memories = store.search(namespace)
    if memories:
        instructions = memories[0].value
    else:
        instructions = ""

    system_msg = MODEL_SYSTEM_MESSAGE.format(user_profile= user_profile, todo=todo, instructions=instructions)

    response = model.bind_tools([UpdateMemory], parallel_tool_calls=False).invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": [response]}

def update_profile(state: MessagesState, config: RunnableConfig, store: BaseStore):
                                                                                  



















